# Reproduction of deuteron calculations (1801.03897)

In [1]:
import numpy as np
np.warnings.filterwarnings('ignore')

from scipy.optimize import minimize

from itertools import product

# Everything we need from OpenFermion
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner
from openfermion.transforms import get_sparse_operator
from openfermion.utils import get_ground_state

# Everything we need from Qiskit
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute, BasicAer
from qiskit.visualization import plot_state_paulivec
from qiskit.quantum_info.operators.pauli import pauli_group, Pauli

## Step 1: Hamiltonians

In [2]:
# Specify some of the required constants and functions; units are in MeV
V0 = -5.68658111
ħω = 7

# Kinetic energy coefficients <n'|T|n>
def T(n, n_prime):
    delta = int(n == n_prime)
    delta_p1 = int(n == (n_prime + 1))
    delta_m1 = int(n == (n_prime - 1))
    
    return (ħω/2) * ((2*n + 1.5)*delta - np.sqrt(n*(n+0.5))*delta_p1 - np.sqrt((n+1)*(n+1.5))*delta_m1)

# Potential energy coefficients <n'|V|n> = V_0 δ(n, 0) δ(n, n')
def V(n, n_prime): 
    return V0 * int((n == 0) and (n == n_prime))

In [15]:
# Construct the Fermionic Hamiltonian
def H(N, jw=True):
    # T and V for the 0th term are constant
    H = FermionOperator('1^ 1', V(0, 0) + T(0, 0))
    
    for n, n_prime in product(range(N), repeat=2):
        if n == 0 and n_prime == 0: # Already made this one
            continue
            
        H += FermionOperator(f"{n_prime+1}^ {n+1}", V(n, n_prime) + T(n, n_prime))
        
    if not jw:
        return H
    
    return jordan_wigner(H)

In [4]:
H(2, jw=False)

-0.43658110999999966 [1^ 1] +
-4.286607049870561 [1^ 2] +
-4.286607049870561 [2^ 1] +
12.25 [2^ 2]

In [5]:
get_sparse_operator(H(2)).toarray().shape

(8, 8)

In [6]:
H_test = np.array([[-0.4366, -4.2866, 0, 0],
             [-4.2866, 12.25, -7.8262, 0],
             [0, -7.8262, 19.25, -11.3413],
              [0, 0, -11.3413, 26.25]])

In [7]:
np.linalg.eigvals(H_test)

array([-2.14398753,  6.07153343, 17.7516723 , 35.6341818 ])

In [8]:
ID = np.eye(2) / np.sqrt(2)
X = np.array([[0, 1], [1, 0]]) / np.sqrt(2)
Z = np.array([[1, 0], [0, -1]])/ np.sqrt(2) 
Y = np.array([[0, -1j], [1j, 0]]) / np.sqrt(2)

In [9]:
paulis = {"II" : np.kron(ID, ID),
         "IZ" : np.kron(ID, Z), 
         "ZI" : np.kron(Z, ID),
         "ZZ" : np.kron(Z, Z),
         "IX" : np.kron(ID, X),
         "XI" : np.kron(X, ID),
         "XX" : np.kron(X, X), 
         "XZ" : np.kron(X, Z),
         "ZX" : np.kron(Z, X),
         "YY" : np.kron(Y, Y)}

In [10]:
coeffs = {} 

for label, mat in paulis.items():
    print(f"{label} : {np.trace(np.dot(H_test, mat))}")
    coeffs[label] = np.trace(np.dot(H_test, mat))

II : 28.656699999999994
IZ : -9.843299999999996
ZI : -16.843299999999996
ZZ : -2.8432999999999993
IX : -15.627899999999997
XI : 0.0
XX : -7.826199999999998
XZ : 0.0
ZX : 7.0546999999999995
YY : (-7.826199999999998+0j)


In [11]:
acc_H = np.zeros((4, 4))

for label, mat in paulis.items():
    acc_H += np.trace(np.dot(H_test, mat)).real * mat.real

In [12]:
acc_H

array([[ -0.4366,  -4.2866,   0.    ,   0.    ],
       [ -4.2866,  12.25  ,  -7.8262,   0.    ],
       [  0.    ,  -7.8262,  19.25  , -11.3413],
       [  0.    ,   0.    , -11.3413,  26.25  ]])

In [13]:
paulis["IX"]

array([[0. , 0.5, 0. , 0. ],
       [0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5],
       [0. , 0. , 0.5, 0. ]])

## Step 2: Exact diagonalization

In [16]:
# Pretty quick up to N ~ 15 regime 
for N in range(1, 10):
    H_N = H(N)
    E_N = get_ground_state(get_sparse_operator(H_N))[0]
    print(f"Ground state energy of H_{N} is {E_N} MeV")

Ground state energy of H_1 is -0.4365811099999998 MeV
Ground state energy of H_2 is -1.7491598763215317 MeV
Ground state energy of H_3 is -2.045670898406443 MeV
Ground state energy of H_4 is -2.1439810307998672 MeV
Ground state energy of H_5 is -2.1835917100257674 MeV
Ground state energy of H_6 is -2.2015681487933088 MeV
Ground state energy of H_7 is -2.210415825352825 MeV
Ground state energy of H_8 is -2.215037872268044 MeV
Ground state energy of H_9 is -2.2175664908674637 MeV


## Step 3: Coupled cluster circuits

In [17]:
# Straight from Fig. 1
# Circuits are different for N = 2 and N = 3
def efficient_circuit(t1, t2, t3):
    c = ClassicalRegister(2)
    q = QuantumRegister(2)
    
    circuit = QuantumCircuit(q, c)
    
    circuit.ry(t1, q[0])
    circuit.cu3(t2, 0, 0, q[0], q[1])
    circuit.cx(q[1], q[0])
    circuit.cu3(t3, 0, 0, q[1], q[0])
    
    return circuit

## Step 4: Variational quantum eigensolver

In [18]:
# Set up the backend to be the state vector simulator
backend = BasicAer.get_backend('statevector_simulator')

# Runs a circuit to compute the expectation value of the energy given current parameters
def compute_energy(params, N):
    energy = 0
    
    # Create the coupled-cluster circuit and get the output state 
    circuit = efficient_circuit(*list(params))
        
    # Run the circuit and get the output state
    result = execute(circuit, backend).result()
    psi = result.get_statevector(circuit)
    rho = np.einsum("i,j->ij", psi, psi)
    
    # Calculate the energy by calculating the expectation value 
    # of each Pauli
    for pauli, coeff in coeffs.items():
        # Calculate the expectation value
        exp_val = np.real(np.trace(np.dot(rho, paulis[pauli])))

        energy += coeff * exp_val
        
    return energy

In [19]:
N= 2
# Initialize guesses for parameters and run VQE
t = np.random.randn(3)

minimize(compute_energy, np.array([t]), args=(N), method='CG', options={'maxiter' : 20})

     fun: -2.143987534364211
     jac: array([ 1.16229057e-06, -9.44733620e-06,  1.90734863e-06])
 message: 'Optimization terminated successfully.'
    nfev: 155
     nit: 17
    njev: 31
  status: 0
 success: True
       x: array([0.83783217, 0.92685472, 0.76002402])

## 3 qubit version

In [20]:
# Straight from Fig. 1
# Circuits are different for N = 2 and N = 3
def efficient_circuit_3(t1, t2, t3, t4, t5, t6, t7):
    c = ClassicalRegister(2)
    q = QuantumRegister(2)
    
    circuit = QuantumCircuit(q, c)
    
    circuit.ry(t1, q[0])
    circuit.cu3(t2, 0, 0, q[0], q[1])
    circuit.cx(q[1], q[0])
    circuit.cu3(t3, 0, 0, q[1], q[0])
    
    return circuit